In [1]:
import pandas as pd
import numpy as np
import os

In [2]:
os.listdir(os.getcwd())

['.ipynb_checkpoints',
 'clean_data.csv',
 'clean_data.ipynb',
 'source',
 'test-balanced.csv',
 'train-balanced.csv']

In [3]:
#read the training data from the csv file
header = ['label','comment']
data = pd.read_table('train-balanced.csv',
                    sep='\t', 
                    delimiter=',', 
                    names=header,
                    usecols=[0,1],
                    dtype={'label':int,'comment':str},
                    keep_default_na=False)

In [4]:
#helper function to remove the # from the comments
def helper(user_comment):
    if '#' in user_comment:
        user_comment = [term.replace('#','') for term in user_comment]
    return user_comment    
clean_data = data.comment.apply(helper)        

In [5]:
#write the cleaned data into a csv file
clean_data.to_csv('clean_data.csv',
           sep= '|')